In [900]:
import pandas as pd
import numpy as np
import lxml
from geopy.geocoders import Nominatim
import time
import math

In [901]:
df = pd.read_csv(r'AccidentesAviones.csv')


In [902]:
#Renombrando columnas 
df = df.rename(columns={'Unnamed: 0': 'Id','fecha':'Fecha','HORA declarada':'HoraDeclarada','OperadOR':'OperadorAerolinea','flight_no':'NumeroVuelo','Ruta':'LugarAccidente','route':'RutaVuelo','ac_type':'TipoAeronave','registration':'Matricula','all_aboard':'TotalABordo','PASAJEROS A BORDO':'PasajerosABordo','crew_aboard':'TripulacionABordo','cantidad de fallecidos':'CantidadFallecidos','passenger_fatalities':'MuertesPasajeros','crew_fatalities':'MuertesTripulacion','ground':'Tierra', 'cn_ln':'SerieConstruccion'})


In [903]:
#reemplazar signo de interrogacion por NA
df = df.replace('?', None)

In [904]:
#Mirando Nulos
df.isnull().sum()

Id                       0
Fecha                    0
HoraDeclarada         1504
LugarAccidente           5
OperadorAerolinea       10
NumeroVuelo           3682
RutaVuelo              762
TipoAeronave            13
Matricula              272
SerieConstruccion      667
TotalABordo             17
PasajerosABordo        221
TripulacionABordo      219
CantidadFallecidos       8
MuertesPasajeros       235
MuertesTripulacion     235
Tierra                  44
summary                 59
dtype: int64

In [905]:
#Eliminando duplicados
df = df.drop_duplicates()

In [906]:
# viendo cuantos nulos hay por columna
print('Porcentaje de valores faltantes de la variable Hora Declarada:', (df.HoraDeclarada.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable Ruta Vuelo:', (df.RutaVuelo.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable Operador/Aerolinea:', (df.OperadorAerolinea.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable NumeroVuelo:', (df.NumeroVuelo.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable LugarAccidente:', (df.LugarAccidente.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable TipoAeronave:', (df.TipoAeronave.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable Matricula:', (df.Matricula.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable SerieConstruccion:', (df.SerieConstruccion.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable TotalABordo:', (df.TotalABordo.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable PasajerosABordo:', (df.PasajerosABordo.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable TripulacionABordo:', (df.TripulacionABordo.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable CantidadFallecidos:', (df.CantidadFallecidos.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable MuertesPasajeros:', (df.MuertesPasajeros.isnull().sum()/len(df)) * 100)
print('Porcentaje de valores faltantes de la variable MuertesTripulacion:', (df.MuertesTripulacion.isnull().sum()/len(df)) * 100)

Porcentaje de valores faltantes de la variable Hora Declarada: 30.031948881789138
Porcentaje de valores faltantes de la variable Ruta Vuelo: 15.215654952076676
Porcentaje de valores faltantes de la variable Operador/Aerolinea: 0.1996805111821086
Porcentaje de valores faltantes de la variable NumeroVuelo: 73.5223642172524
Porcentaje de valores faltantes de la variable LugarAccidente: 0.0998402555910543
Porcentaje de valores faltantes de la variable TipoAeronave: 0.2595846645367412
Porcentaje de valores faltantes de la variable Matricula: 5.431309904153355
Porcentaje de valores faltantes de la variable SerieConstruccion: 13.318690095846645
Porcentaje de valores faltantes de la variable TotalABordo: 0.33945686900958466
Porcentaje de valores faltantes de la variable PasajerosABordo: 4.4129392971246
Porcentaje de valores faltantes de la variable TripulacionABordo: 4.373003194888179
Porcentaje de valores faltantes de la variable CantidadFallecidos: 0.1597444089456869
Porcentaje de valores fa

In [907]:
#corrigiendo la columna hora
df['HoraDeclarada']=df['HoraDeclarada'].fillna('00:00')

In [908]:
def corr_hora(valor):
    if ':' not in valor:
        return f'{valor[:2]}:{valor[-2:]}'
    if valor == None:
        return '00:00'
    else: return valor

In [909]:
df['HoraDeclarada'] = df['HoraDeclarada'].apply(lambda x: corr_hora(x))

In [910]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [911]:
df['TipoAeronave'].value_counts()

Douglas DC-3                                333
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                70
Douglas C-47                                 64
Douglas DC-4                                 41
                                           ... 
Vickers Viscount 759D                         1
H-21B                                         1
Avro 685 York C1                              1
Piper PA-24-250 Comanche                      1
Antonov An 26B-100                            1
Name: TipoAeronave, Length: 2468, dtype: int64

In [912]:
df['summary'][3]

'The first fatal airplane accident in Canada occurred when American barnstormer, John M. Bryant, California aviator was killed.'

In [913]:
dfp = df['LugarAccidente'].str.split(',', expand=True)
dfp = dfp.drop([0,2,3], axis=1)
dfp = dfp.rename(columns={1:'PaisAccidente'})
df = pd.concat([dfp,df], axis= 1)
df.head()

,PaisAccidente,Id,Fecha,HoraDeclarada,LugarAccidente,OperadorAerolinea,NumeroVuelo,RutaVuelo,TipoAeronave,Matricula,SerieConstruccion,TotalABordo,PasajerosABordo,TripulacionABordo,CantidadFallecidos,MuertesPasajeros,MuertesTripulacion,Tierra,summary
0,Virginia,0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,Wright Flyer III,None,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,France,1,1909-09-07,00:00,"Juvisy-sur-Orge, France",None,None,Air show,Wright Byplane,SC1,None,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,New Jersey,2,1912-07-12,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,None,Test flight,Dirigible,None,None,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,British Columbia,3,1913-08-06,00:00,"Victoria, British Columbia, Canada",Private,None,None,Curtiss seaplane,None,None,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,None,4,1913-09-09,18:30,Over the North Sea,Military - German Navy,None,None,Zeppelin L-1 (airship),None,None,20,None,None,14,None,None,0,The airship flew into a thunderstorm and encou...


In [914]:
print(df)

          PaisAccidente    Id      Fecha HoraDeclarada  \
0              Virginia     0 1908-09-17         17:18   
1                France     1 1909-09-07         00:00   
2            New Jersey     2 1912-07-12         06:30   
3      British Columbia     3 1913-08-06         00:00   
4                  None     4 1913-09-09         18:30   
...                 ...   ...        ...           ...   
5003             Alaska  5003 2021-03-28         18:35   
5004            Nigeria  5004 2021-05-21         18:00   
5005            Myanmar  5005 2021-06-10         08:00   
5006               Sulu  5006 2021-07-04         11:30   
5007             Russia  5007 2021-07-06         15:00   

                          LugarAccidente                OperadorAerolinea  \
0                    Fort Myer, Virginia             Military - U.S. Army   
1                Juvisy-sur-Orge, France                             None   
2              Atlantic City, New Jersey             Military - U.S. Nav

In [915]:
dfp

,PaisAccidente
0,Virginia
1,France
2,New Jersey
3,British Columbia
4,None
...,...
5003,Alaska
5004,Nigeria
5005,Myanmar
5006,Sulu


In [916]:
dfp.isnull().sum()

PaisAccidente    181
dtype: int64

1. Webscrapping de las paginas web para los datasets:
- atc = Errores control trafico aereo
- aves = golpes de aves
- inc = incendios en bodega
- defe = defectos de diseno
- com = falta de combustible
- rel = Relampagos
- sab = Sabotaje - secuestro

2. modificacion cada uno con mismo formato y unirlos en un solo df

In [917]:
# hacemos EDA, separo el pais donde fue el accidente y lo uno al dataframe,
# ademas agrego una columna para saber a que categoria de la causa del accidente posterior a la union de los dataframes
atc = pd.read_html('http://www.planecrashinfo.com/CATATC.htm')
atc = atc[0].dropna().drop(0,axis=0).drop(3, axis=1)
atc = atc.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
atc['CausaAccidente'] = 'Errores Control Trafico Aereo'
atc2 = atc['LugarAccidente'].str.split(',', expand=True)
atc2 = atc2.drop([0],axis=1)
atc = pd.concat([atc2, atc], axis=1)
atc = atc.rename( columns= {1:'PaisAccidente'})
atc


,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Virginia,6/13/1947,"Leesburg, Virginia",Pennsylvania AL,Errores Control Trafico Aereo
2,Russia,05/22/1948,"Khabarovsk, Russia",Aeroflot,Errores Control Trafico Aereo
3,Virginia,11/01/1949,"Arlington, Virginia",Eastern Air Lines,Errores Control Trafico Aereo
4,Russia,10/05/1952,"Skvoritsy, Russia",Aeroflot,Errores Control Trafico Aereo
5,Spain,04/14/1958,"Castel de Fels, Spain",Aviaco,Errores Control Trafico Aereo
6,Alaska,07/21/1961,"Shemya, Alaska",Alaska AL,Errores Control Trafico Aereo
7,New York,02/08/1965,"New York, New York",Eastern AL,Errores Control Trafico Aereo
8,Ohio,03/09/1967,"Urbana, Ohio",Trans World Airlines,Errores Control Trafico Aereo
9,Puerto Rico,03/05/1969,"San Juan, Puerto Rico",Prinair,Errores Control Trafico Aereo
10,Indiana,09/09/1969,"Fairland, Indiana",Allegheny AL,Errores Control Trafico Aereo


In [918]:
aves = pd.read_html('http://www.planecrashinfo.com/CATBS.htm')
aves = aves[0]
aves = aves.drop([0],axis=0).drop([3],axis=1)
aves = aves.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
aves['CausaAccidente'] = 'Golpes De Aves'
aves2 = aves['LugarAccidente'].str.split(',', expand=True)
aves2 = aves2.drop([0],axis=1)
aves = pd.concat([aves2, aves], axis=1)
aves = aves.rename( columns= {1:'PaisAccidente'})
aves

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Massachusetts,10/04/1960,"Boston, Massachusetts",Eastern AL,Golpes De Aves
2,Maryland,11/23/1962,"Ellicott, Maryland",United AL,Golpes De Aves
3,Ireland,03/24/1968,"Wexford Harbor, Ireland",Air Lingus,Golpes De Aves
4,Ethiopia,09/15/1988,"Bahar Dar, Ethiopia",Ethiopian AL,Golpes De Aves
5,None,04/18/1990,Off Panama,Aero Perlas,Golpes De Aves
6,Alaska,09/22/1995,"Anchorage, Alaska",U.S. Air Force,Golpes De Aves
7,Congo,04/19/2000,"Pepo, Congo",Centrafricain Airlines,Golpes De Aves
8,New York,01/15/2009,"New York, New York",US Airways,Golpes De Aves


In [919]:
inc = pd.read_html('http://www.planecrashinfo.com/CATCH.htm')
inc = inc[0]
inc = inc.drop([0],axis=0).drop([3],axis=1)
inc = inc.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
inc['CausaAccidente'] = 'Incendios En Bodega'
inc2 = inc['LugarAccidente'].str.split(',', expand=True)
inc2 = inc2.drop([0],axis=1)
inc = pd.concat([inc2, inc], axis=1)
inc = inc.rename( columns= {1:'PaisAccidente'})
inc = inc.drop([2], axis=1)
inc

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Picrdie,05/14/1923,"Amiens, Picrdie, France",Air Union,Incendios En Bodega
2,Brazil,05/07/1928,"Florianopolis, Brazil","Florianopolis, Brazil",Incendios En Bodega
3,Belgium,03/28/1933,"Dixmude, Belgium",Imperial Airways,Incendios En Bodega
4,Belgium,07/28/1937,"Hal, Belgium",KLM,Incendios En Bodega
5,Romania,08/06/1937,"Bistrita, Romania",Aeroflot,Incendios En Bodega
...,...,...,...,...,...
59,None,08/03/2008,Vancouver Island,Pacific Coastal Airlines,Incendios En Bodega
60,Congo,08/06/2009,"Nganga Lingolo, Congo",Aero Fret Business,Incendios En Bodega
61,Philippines,04/21/2010,"Pampanga, Philippines",Interisland Airlines,Incendios En Bodega
62,United Arab Emirates,09/03/2010,"Dubai, United Arab Emirates",United Parcel Service- UPS,Incendios En Bodega


In [920]:
defe = pd.read_html('http://www.planecrashinfo.com/CATDF.htm')
defe = defe[0]
defe = defe.drop([0],axis=0).drop([3],axis=1)
defe = defe.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
defe['CausaAccidente'] = 'Defectos De Disenio'
defe2 = defe['LugarAccidente'].str.split(',', expand=True)
defe2 = defe2.drop([0],axis=1)
defe = pd.concat([defe2, defe], axis=1)
defe = defe.rename( columns= {1:'PaisAccidente'})
#inc = inc.drop([2], axis=1)
defe

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Kansas,03/31/1931,"Bazaar, Kansas",Trans Cont. & West AW,Defectos De Disenio
2,Utah,10/24/1947,"Bryce Canyon, Utah",United AL,Defectos De Disenio
3,New Mexico,11/11/1947,"Gallup, New Mexico",American AL,Defectos De Disenio
4,Pennsylvania,06/17/1948,"Mt. Carmel, Pennsylvania",United AL,Defectos De Disenio
5,Minnesota,08/29/1948,"Winona, Minnesota",Northwest Orient AL,Defectos De Disenio
6,Italy,01/10/1954,"Elba, Italy",British Overseas AW,Defectos De Disenio
7,Italy,04/08/1954,"Off Stromboli, Italy",Trans Canada AL,Defectos De Disenio
8,Nigeria,02/05/1955,"Calabar, Nigeria",West African AW,Defectos De Disenio
9,Texas,09/29/1959,"Buffalo, Texas",Braniff AL,Defectos De Disenio
10,Indiana,03/17/1960,"Tell City, Indiana",Northwest Orient AL,Defectos De Disenio


In [921]:
com = pd.read_html('http://www.planecrashinfo.com/CATFS.htm')
com = com[0]
com = com.drop([0],axis=0).drop([3],axis=1)
com = com.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
com['CausaAccidente'] = 'Falta De Combustible'
com2 = com['LugarAccidente'].str.split(',', expand=True)
com2 = com2.drop([0],axis=1)
com = pd.concat([com2, com], axis=1)
com = com.rename( columns= {1:'PaisAccidente'})
com = com.drop([2], axis=1)
com

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,None,05/18/1935,Knowles Flying Service,"Flint, Michigan",Falta De Combustible
2,None,12/31/1935,Imperial Airways,"Alexandria, Egypt",Falta De Combustible
3,New Guinea,07/02/1937,"Lae, New Guinea",Purdue Res. Found.,Falta De Combustible
4,Calif.,11/29/1938,"Off Point Reyes, Calif.",United Air Lines,Falta De Combustible
5,Newfoundland,02/09/1943,"Gander, Newfoundland",British Overseas AW,Falta De Combustible
6,Michigan,12/28/1946,"Michigan City, Michigan",American AL,Falta De Combustible
7,New Jersey,01/05/1947,"Carmel, New Jersey",Nationwide Air Trans.,Falta De Combustible
8,England,01/11/1947,"Lympne, England",BOAC,Falta De Combustible
9,Georgia,01/07/1948,"Savannah, Georgia",Coastal Air Lines,Falta De Combustible
10,None,01/30/1948,Near Bermuda,British So. Am. AW,Falta De Combustible


In [922]:
rel = pd.read_html('http://www.planecrashinfo.com/CATL.htm')
rel = rel[0]
rel = rel.drop([0],axis=0).drop([3],axis=1)
rel = rel.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
rel['CausaAccidente'] = 'Relampagos'
rel2 = rel['LugarAccidente'].str.split(',', expand=True)
rel2 = rel2.drop([0],axis=1)
rel = pd.concat([rel2, rel], axis=1)
rel = rel.rename( columns= {1:'PaisAccidente'})
#com = com.drop([2], axis=1)
rel

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,New Mexico,09/03/1929,"Mt. Taylor, New Mexico",Trans Con. Air Transport,Relampagos
2,Belgium,07/28/1937,"Hal, Belgium",KLM Royal Dutch Airlines,Relampagos
3,Romania,07/22/1938,"Stulpica, Romania",LOT,Relampagos
4,Virginia,08/31/1940,"Lovettsville, Virginia",Penn Central AL,Relampagos
5,Brazil,09/27/1946,"Rio Doce, Brazil",Panair do Brasil,Relampagos
6,Russia,05/13/1949,"Novosibirsk, Russia",Aeroflot,Relampagos
7,Italy,01/17/1951,"Civitavecchia, Italy",Alitalia,Relampagos
8,Russia,09/07/1951,"Kazakh, Russia",Aeroflot,Relampagos
9,Italy,06/26/1959,"Varese, Italy",Trans World AL,Relampagos
10,Brazil,07/19/1961,"Azul, Brazil",Aerolineas Argentinas,Relampagos


In [923]:
sab = pd.read_html('http://www.planecrashinfo.com/CATS.htm')
sab = sab[1]
sab = sab.drop([0],axis=0).drop([3],axis=1)
sab = sab.rename(columns={0:'Fecha',1:'LugarAccidente',2:'Operador/Aerolinea'})
sab['CausaAccidente'] = 'Sabotaje - Secuestro'
sab2 = sab['LugarAccidente'].str.split(',', expand=True)
sab2 = sab2.drop([0],axis=1)
sab = pd.concat([sab2, sab], axis=1)
sab = sab.rename( columns= {1:'PaisAccidente'})
sab = sab.drop([2], axis=1)
sab

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Belgium,03/28/1933,"Dixmude, Belgium",Imperial AW,Sabotaje - Secuestro
2,None,1939 - 1945,-,-,Sabotaje - Secuestro
3,None,07/16/1948,Pacific Ocean,Cathay Pacific AW,Sabotaje - Secuestro
4,NaN,NaN,NaN,NaN,Sabotaje - Secuestro
5,Philippines,05/07/1949,"Sibuyan Sea, Philippines",Philippine AL,Sabotaje - Secuestro
...,...,...,...,...,...
113,Ukraine,07/17/2014,"Hrabove, Ukraine",Malaysia Airlines,Sabotaje - Secuestro
114,Fran,03/24/2015,"Digne-Les-Bains, Fran",Germanwings,Sabotaje - Secuestro
115,Egypt,10/31/2015,"Hasna, Egypt",Metrojet,Sabotaje - Secuestro
116,Somalia,02/02/2016,"Mogadishu, Somalia",Daallo Airlines,Sabotaje - Secuestro


Union de dataframes en uno solo llamado df2 que es el dataframe de causas de accidentes por paises

In [924]:
df2 = pd.concat([atc,aves,inc,defe,com,rel,sab], axis=0)
df2.head()

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Virginia,6/13/1947,"Leesburg, Virginia",Pennsylvania AL,Errores Control Trafico Aereo
2,Russia,05/22/1948,"Khabarovsk, Russia",Aeroflot,Errores Control Trafico Aereo
3,Virginia,11/01/1949,"Arlington, Virginia",Eastern Air Lines,Errores Control Trafico Aereo
4,Russia,10/05/1952,"Skvoritsy, Russia",Aeroflot,Errores Control Trafico Aereo
5,Spain,04/14/1958,"Castel de Fels, Spain",Aviaco,Errores Control Trafico Aereo


Eliminamos columnas con demasiados nulos con mas del 15% de nulos y luego eliminamos nulos del dataframe, esto en este orden para no perder informacion relevante:
- Hora Declarada: 30.031948881789138
- Ruta Vuelo: 15.215654952076676
- Operador/Aerolinea: 0.1996805111821086
- NumeroVuelo: 73.5223642172524
- LugarAccidente: 0.0998402555910543
- TipoAeronave: 0.2595846645367412
- Matricula: 5.431309904153355
- SerieConstruccion: 13.318690095846645
- TotalABordo: 0.33945686900958466
- PasajerosABordo: 4.4129392971246
- TripulacionABordo: 4.373003194888179
- CantidadFallecidos: 0.1597444089456869
- MuertesPasajeros: 4.692492012779552
- MuertesTripulacion: 4.692492012779552

Basado en esta informacion eliminamos:
- Hora declarada
- NumeroVuelo
- RutaVuelo

In [925]:
df = df.drop(['HoraDeclarada','NumeroVuelo','RutaVuelo'], axis=1)
df.head()

,PaisAccidente,Id,Fecha,LugarAccidente,OperadorAerolinea,TipoAeronave,Matricula,SerieConstruccion,TotalABordo,PasajerosABordo,TripulacionABordo,CantidadFallecidos,MuertesPasajeros,MuertesTripulacion,Tierra,summary
0,Virginia,0,1908-09-17,"Fort Myer, Virginia",Military - U.S. Army,Wright Flyer III,None,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,France,1,1909-09-07,"Juvisy-sur-Orge, France",None,Wright Byplane,SC1,None,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,New Jersey,2,1912-07-12,"Atlantic City, New Jersey",Military - U.S. Navy,Dirigible,None,None,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,British Columbia,3,1913-08-06,"Victoria, British Columbia, Canada",Private,Curtiss seaplane,None,None,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,None,4,1913-09-09,Over the North Sea,Military - German Navy,Zeppelin L-1 (airship),None,None,20,None,None,14,None,None,0,The airship flew into a thunderstorm and encou...


In [926]:

print(df.shape,
df2.shape)

(5008, 16) (317, 5)


In [927]:
#Eliminamos Nulos
df = df.dropna()
df2 = df2.dropna()

In [928]:
print(df.shape,
df2.shape)

(4087, 16) (285, 5)


resumen hasta ahora:
EDA 2 databases:
- Accidentes aviones = df
- Causas Accidentes = df1

-vamos a la tercera base que es basado en aerolineas y los accidentes a lo largo de la historia
Bases aerolineas (civiles) 25 en total que vamos a volver una sola (df3):
- a
- a1
- a2
- a3
- a4
- b
- c
- c2
- d
- f
- h
- j
- l
- l2
- m 
- n
- p
- p2
- r
- s
- s2
- t
- t2
- u
- z


In [929]:
a = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Aj-Ar.htm')
a = a[0]
a = a.drop([0],axis=0)
a = a.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
a2 = a['LugarAccidente'].str.split(',', expand=True)
a2 = a2.drop([0],axis=1)
a = pd.concat([a2, a], axis=1)
a = a.rename( columns= {1:'PaisAccidente'})
a = a.drop([2,3], axis=1)
a['Fecha'] = pd.to_datetime(a['Fecha'])
a

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Russia,AKF Polet,1991-06-26,"Gulf of Finland, Russia",Antonov AN-24
2,Turkmenistan,Akhal Air,1996-03-12,"Karabogazgol Bay, Turkmenistan",Mil Mi-8T (helicopter)
3,Italy,Ala Littolia,1940-12-24,"Capri Island, Italy",Savoia-Marchetti SM.75
4,Egypt,Ala Littoria,1935-08-07,"Cairo, Egypt",Savoia-Marchetti S-66
5,Sudan,Ala Littoria,1937-08-02,"Wadi Halfa, Sudan",Savoia-Marchetti SM73
...,...,...,...,...,...
179,Russia,Armavia,2006-05-03,"Off Adler, Russia",Airbus A320-211
180,California,Arrow Airlines,1949-12-07,"Vallejo, California",Douglas DC-3
181,Newfoundland,Arrow Airways,1985-12-12,"Gander, Newfoundland, Canada",McDonnell Douglas DC-8 Super 63PF
182,Russia,Arsenyev Aviation Production Association,1994-09-30,"Chaibukha, Russia",Antonov AN-8


In [930]:
a1 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Aeroflot.htm')
a1 = a1[0]
a1 = a1.drop([0],axis=0)
a1 = a1.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
a12 = a1['LugarAccidente'].str.split(',', expand=True)
a12 = a12.drop([0],axis=1)
a1 = pd.concat([a12, a1], axis=1)
a1 = a1.rename( columns= {1:'PaisAccidente'})
a1 = a1.drop([2], axis=1)
a1['Fecha'] = pd.to_datetime(a1['Fecha'])
a1

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,None,Aeroflot,1934-12-31,?,Tupolev ANT-9
2,Tajikistan,Aeroflot,1936-07-28,"Kulyab, Tajikistan",Tupolev ANT-9
3,Romania,Aeroflot,1937-08-06,"Bistrita, Romania",Douglas DC-2
4,Turkmenistan,Aeroflot,1941-11-29,"Chardzhou, Turkmenistan",Tupolev ANT-6
5,Kazakhstan,Aeroflot,1941-12-26,"Baiserke, Kazakhstan",Tupolev ANT-6
...,...,...,...,...,...
258,Georgia,Aeroflot / Aeroflot,1982-08-14,"Sukhumi, Georgia, USSR",Let 410UVP Turbojet / Tupolev TU-134V
259,Kazakhstan,Aeroflot / Kazakhstan,1965-01-03,"Alma-Ata, Kazakhstan",Ilyushin IL-18B
260,Ukraine,Aeroflot / Soviet Air Force,1985-05-03,"Near L'vov, Ukraine, USSR",Tupolev TU-134A / Antonov An-26
261,Russia,Aeroflot Russian International Airlines,2019-05-05,"Moscow, Russia",Sukhoi Superjet-100-95B


In [931]:
a2 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Af-Ai.htm')
a2 = a2[0]
a2 = a2.drop([0],axis=0)
a2 = a2.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
a22 = a2['LugarAccidente'].str.split(',', expand=True)
a22 = a22.drop([0],axis=1)
a2 = pd.concat([a22, a2], axis=1)
a2 = a2.rename( columns= {1:'PaisAccidente'})
a2 = a2.drop([2], axis=1)
a2['Fecha'] = pd.to_datetime(a2['Fecha'])
a2

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,None,Afghan Army,2006-07-27,Afghanistan,Mil Mi-8
2,Afghanistan,Afghan Army,2009-01-15,"Adraskan district, Afghanistan",Mil Mi-17
3,DR Congo,Africa One Congo,2007-10-04,"Kinshasa, DR Congo",Antonov AN-26
4,Zaire,African Air,1996-01-08,"Kinshasa, Zaire",Antonov AN-32B
5,Kenya,African Commuter Services,2003-01-24,"Busia, Kenya",Grumman G-159 Gulfstream I
...,...,...,...,...,...
291,Maine,Airwave Transport,2000-07-19,"Linneus, Maine",Grumman G-159 Gulfstream I
292,India,Airways,1953-04-14,"Near Guwahati, India",Douglas C-47A
293,BC,AirWest Airlines,1978-09-03,"Coal Harbour, BC, Canada",de havilland Canada Twin Otter 200
294,Italy,Airwork,1952-08-25,"Off Trapani, Italy",Handley Page HP-81 Hermes 4A


In [932]:
a3 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Aj-Ar.htm')
a3 = a3[0]
a3 = a3.drop([0],axis=0)
a3 = a3.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
a32 = a3['LugarAccidente'].str.split(',', expand=True)
a32 = a32.drop([0],axis=1)
a3 = pd.concat([a32, a3], axis=1)
a3 = a3.rename( columns= {1:'PaisAccidente'})
a3 = a3.drop([2,3], axis=1)
a3['Fecha'] = pd.to_datetime(a3['Fecha'])
a3

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Russia,AKF Polet,1991-06-26,"Gulf of Finland, Russia",Antonov AN-24
2,Turkmenistan,Akhal Air,1996-03-12,"Karabogazgol Bay, Turkmenistan",Mil Mi-8T (helicopter)
3,Italy,Ala Littolia,1940-12-24,"Capri Island, Italy",Savoia-Marchetti SM.75
4,Egypt,Ala Littoria,1935-08-07,"Cairo, Egypt",Savoia-Marchetti S-66
5,Sudan,Ala Littoria,1937-08-02,"Wadi Halfa, Sudan",Savoia-Marchetti SM73
...,...,...,...,...,...
179,Russia,Armavia,2006-05-03,"Off Adler, Russia",Airbus A320-211
180,California,Arrow Airlines,1949-12-07,"Vallejo, California",Douglas DC-3
181,Newfoundland,Arrow Airways,1985-12-12,"Gander, Newfoundland, Canada",McDonnell Douglas DC-8 Super 63PF
182,Russia,Arsenyev Aviation Production Association,1994-09-30,"Chaibukha, Russia",Antonov AN-8


In [933]:
a4 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20As-Az.htm')
a4 = a4[0]
a4 = a4.drop([0],axis=0)
a4 = a4.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
a42 = a4['LugarAccidente'].str.split(',', expand=True)
a42 = a42.drop([0],axis=1)
a4 = pd.concat([a42, a4], axis=1)
a4 = a4.rename( columns= {1:'PaisAccidente'})
a4 = a4.drop([2,3], axis=1)
a4['Fecha'] = pd.to_datetime(a4['Fecha'])
a4

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Russia,AKF Polet,1991-06-26,"Gulf of Finland, Russia",Antonov AN-24
2,Turkmenistan,Akhal Air,1996-03-12,"Karabogazgol Bay, Turkmenistan",Mil Mi-8T (helicopter)
3,Italy,Ala Littolia,1940-12-24,"Capri Island, Italy",Savoia-Marchetti SM.75
4,Egypt,Ala Littoria,1935-08-07,"Cairo, Egypt",Savoia-Marchetti S-66
5,Sudan,Ala Littoria,1937-08-02,"Wadi Halfa, Sudan",Savoia-Marchetti SM73
...,...,...,...,...,...
179,Russia,Armavia,2006-05-03,"Off Adler, Russia",Airbus A320-211
180,California,Arrow Airlines,1949-12-07,"Vallejo, California",Douglas DC-3
181,Newfoundland,Arrow Airways,1985-12-12,"Gander, Newfoundland, Canada",McDonnell Douglas DC-8 Super 63PF
182,Russia,Arsenyev Aviation Production Association,1994-09-30,"Chaibukha, Russia",Antonov AN-8


In [934]:
b = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20B.htm')
b = b[0]
b = b.drop([0],axis=0)
b = b.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
b2 = b['LugarAccidente'].str.split(',', expand=True)
b2 = b2.drop([0],axis=1)
b = pd.concat([b2, b], axis=1)
b = b.rename( columns= {1:'PaisAccidente'})
b = b.drop([2,3], axis=1)
b['Fecha'] = pd.to_datetime(b['Fecha'])
b

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Bolivia,BAFIN,1990-10-19,"Near Bella Vista, Bolivia",Douglas C-47B-40-DK
2,Siberia,Baikal Air,1994-01-03,"Near Mamony, Siberia, Russia",Tupolev TU-154M
3,Afghanistan,Bakhtar Afghan Airlines,1973-04-18,"Bamiyan, Afghanistan",de Havilland Canada DHC-6 Twin Otter 100
4,Afghanistan,Bakhtar Afghan Airlines,1983-03-10,"Near Shach Goan, Afghanistan",de Havilland Canada DHC-6 Twin Otter 300
5,Afghanistan,Bakhtar Afghan Airlines,1987-06-11,"Khost, Afghanistan",Antonov AN-26
...,...,...,...,...,...
190,Arizona,Business Air Connection,1996-02-05,"St. Johns, Arizona",ConvairCV-440
191,Texas,Business Jet Services,2004-11-22,"Houston, Texas",Grumman Gulfstream G3
192,Democratic Republic of Congo,Busy Bee Congo,2019-11-24,"Goma, Democratic Republic of Congo",Dornier 228-201
193,Oregon,Butler Aviation Inc.,1979-09-14,"Near Klamath Falls, Oregon",Douglas DC-7


In [935]:
c = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Ca-Ch.htm')
c = c[0]
c = c.drop([0],axis=0)
c = c.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
c2 = c['LugarAccidente'].str.split(',', expand=True)
c2 = c2.drop([0],axis=1)
c = pd.concat([c2, c], axis=1)
c = c.rename( columns= {1:'PaisAccidente'})
c = c.drop([2], axis=1)
c['Fecha'] = pd.to_datetime(c['Fecha'])
c

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,China,CAAC,1976-01-21,"Near Shanghai, China",Antonov AN-24
2,China,CAAC,1979-03-14,"Bejing, China",Hawker Siddeley Trident 2E
3,Guangxi,CAAC,1982-04-26,"Near Yangshuo, Guangxi, China",Hawker Siddeley Trident 2E
4,China,CAAC,1982-12-24,"Guangzhou, China",Ilyushin IL-18B
5,China,CAAC,1983-09-14,"Guilin, China",Hawker Siddeley Trident 2E / II-28
...,...,...,...,...,...
186,China,China Southern Airlines,1992-11-24,"Near Guilin, China",Boeing B-737-3Y0
187,China,China Southern Airlines,1997-05-08,"Huangtian, China",Boeing B-737-31B
188,China,China Southern Airlines / Xiamen Airlines,1990-10-02,"Guangzhou, China",Boeing B-737-247 / Boeing B-757-21B
189,Sichuan,China Southwest Airlines,1988-01-18,"Near Chungking, Sichuan, China",Ilyushin IL-18D


In [936]:
c2 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Ci-Cz.htm')
c2 = c2[0]
c2 = c2.drop([0],axis=0)
c2 = c2.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
c22 = c2['LugarAccidente'].str.split(',', expand=True)
c22 = c22.drop([0],axis=1)
c2 = pd.concat([c22, c2], axis=1)
c2 = c2.rename( columns= {1:'PaisAccidente'})
c2 = c2.drop([2], axis=1)
c2['Fecha'] = pd.to_datetime(c2['Fecha'])
c2

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Brazil,Cia Brasiliera de Tratores,1982-09-04,"Rio Branco, Brazil",Gates Learjet 25B
2,Hungary,CIDNA,1925-07-24,"Budapest, Hungary",Potez IX
3,France,CIDNA,1925-09-07,"Toul, France",Spad 33
4,France,CIDNA,1927-11-08,"Strasburg, France",Farman F-121 Jabiru
5,France,CIDNA,1928-09-02,"Toul, France",Bleriot Spad 33
...,...,...,...,...,...
117,Guatemala,Cubana de Aviacon,1999-12-21,"Guatemala City, Guatemala",McDonnell Douglas DC-10-30
118,Norway,Cunard Eagle Airways,1961-08-09,"Sola, Norway",Vickers Viking 3B
119,California,Currey Air Transport,1955-09-08,"Burbank, California",Douglas DC-3
120,France,Curtiss-Reid Flying Services Ltd. (Canada),1950-11-13,"Mt. Tete de L'Obiou, France",Douglas C-54B-1-DC


In [937]:
d = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20D-E.htm')
d = d[0]
d = d.drop([0],axis=0)
d = d.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
d2 = d['LugarAccidente'].str.split(',', expand=True)
d2 = d2.drop([0],axis=1)
d = pd.concat([d2, d], axis=1)
d = d.rename( columns= {1:'PaisAccidente'})
d = d.drop([2,3], axis=1)
d['Fecha'] = pd.to_datetime(d['Fecha'])
d

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Somalia,Daallo Airlines,2016-02-02,"Near Mogadishu, Somalia",Airbus A321-111
2,Russia,Dagestan Airlines,2010-12-04,"Moscow, Russia",Tupolev Tu-154M
3,Japan,Dai Nippon,1938-08-10,"Shishijima, Japan",Douglas DF-151
4,Buckinghamshire,Daimler Airways,1923-09-14,"Near Ivinghoe, Buckinghamshire, England",de Havilland DH-34
5,France,Daimler Airways / Grands Express Aeriens,1922-04-07,"Grandvilliers, France",de Havilland DH-18 / Farman F-60 Goliath
...,...,...,...,...,...
240,Michigan,Executive Jet Sales Inc.,1970-05-09,"Pellston, Michigan",Gates Learjet 23
241,Mexico,Exportada De Sal S.A.,1968-12-26,"Ensenada, Mexico",Douglas DC-3
242,Minnesota,Express Airlines - Northwest Airlink,1993-12-01,"Hibbing, Minnesota",British Aerospace Jetstream BA-3100
243,Peru,Expresso Aéreo,1992-09-10,"Bellavista, Peru",Fokker F-27 Friendship 500


In [938]:
f = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20F-G.htm')
f = f[0]
f = f.drop([0],axis=0)
f = f.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
f2 = f['LugarAccidente'].str.split(',', expand=True)
f2 = f2.drop([0],axis=1)
f = pd.concat([f2, f], axis=1)
f = f.rename( columns= {1:'PaisAccidente'})
f = f.drop([2], axis=1)
f['Fecha'] = pd.to_datetime(f['Fecha'])
f

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Colombia,F.R. de la Direccion General de Ados,1980-11-24,"Near Medellin, Colombia",Douglas C-47A-35-DL
2,Cardiff,Fairflight Ltd.,1950-03-12,"Llandow Airport, Cardiff, Wales",Avro 689 Tudor 5
3,Philippines,Fairways (Filipinas Orient Airways),1965-03-08,"Libmanan, Philippines",Douglas DC-3
4,Iraq,Falcon Aviation Group,2008-11-13,"Fallujah, Iraq",Antonov An-12
5,Brazil,FAMA (Argentina),1946-12-23,"Near Rio de Janeiro, Brazil",Avro 685 York I
...,...,...,...,...,...
172,Australia,Guinea Airways Limited,1942-04-21,"Pine Creek, Australia",Lockheed 14 Electra
173,UAE,Gulf Air,1983-09-23,"Near Mina Jebel Ali, UAE",Boeing B-737-2P6
174,Bahrain,Gulf Air,2000-08-23,"Off Manama, Bahrain",Airbus A320-212
175,None,Gulf Aviation (Kalinga Airways),1960-07-10,Persian Gulf,Douglas DC-3


In [939]:
h = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20H-I.htm')
h = h[0]
h = h.drop([0],axis=0)
h = h.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
h2 = h['LugarAccidente'].str.split(',', expand=True)
h2 = h2.drop([0],axis=1)
h = pd.concat([h2, h], axis=1)
h = h.rename( columns= {1:'PaisAccidente'})
h = h.drop([2,3], axis=1)
h['Fecha'] = pd.to_datetime(h['Fecha'])
h

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Alaska,Hageland Aviation Services,1997-04-10,"Near Wainwright, Alaska",Cessna 208B Caravan I
2,Alaska,Hageland Aviation Services,1997-11-08,"Near Barrow, Alaska",Cessna 208B Caravan I
3,Alaska,Hageland Aviation Services Inc.,2003-08-17,"Near Barrow, Alaska",Reims Aviation 406
4,Alaksa,Hageland Aviiation,2016-08-31,"Russian Mission, Alaksa",Cessna 208 Grand Caravan
5,None,Haïti Air Inter,1980-01-02,Haiti,Britten Norman BN Islander
...,...,...,...,...,...
179,Italy,Itavia,1960-10-14,"Elba, Italy",de Havilland DH-114 Heron 2
180,Italy,Itavia,1963-03-30,"Near Sora, Italy",Douglas DC-3
181,Italy,Itavia,1974-01-01,"Near Turin, Italy",Fokker F-28 Fellowship 1000
182,off Ustica,Itavia,1980-06-27,"Tyrrhenian Sea, off Ustica, Italy",McDonnell Douglas DC-9-15


In [940]:
j = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20J-K.htm')
j = j[0]
j = j.drop([0],axis=0)
j = j.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
j2 = j['LugarAccidente'].str.split(',', expand=True)
j2 = j2.drop([0],axis=1)
j = pd.concat([j2, j], axis=1)
j = j.rename( columns= {1:'PaisAccidente'})
j = j.drop([2], axis=1)
j['Fecha'] = pd.to_datetime(j['Fecha'])
j

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Kauai,Jack Harter Helicopters,2003-07-23,"Waialeale,Kauai, Hawaii",Bell 206B
2,India,Jamair,1970-12-05,"Delhi, India",Douglas DC-3
3,India,Jamair,1971-03-26,"Gauhati, India",Douglas DC-3
4,Japan,Japan Air / Japan Flying School,1938-08-24,"Tokyo, Japan",Unknown / Unknown
5,Japan,Japan Air Lines,1952-04-09,"Mihara Volcano, Japan",Martin 202
...,...,...,...,...,...
127,Russia,Kranoyarskie Vozhjushnye Trassy,1942-11-17,"Near Kranoyarsk, Russia",Grazhdansky Vozdushnyi Flot PS-84
128,Russia,Krasnoyarsk Airlines,1992-06-22,"Norlisk, Russia",Antonov AN-12
129,Russia,Krasnoyarskie Avialinii,1996-04-05,"Near Petropavlovsk, Russia",Ilyushin IL-76TD
130,Armenia,Kuban Airlines,1993-12-26,"Gyumri, Armenia",Antonov AN-26


In [941]:
l = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20La-Ln.htm')
l = l[0]
l = l.drop([0],axis=0)
l = l.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
l2 = l['LugarAccidente'].str.split(',', expand=True)
l2 = l2.drop([0],axis=1)
l = pd.concat([l2, l], axis=1)
l = l.rename( columns= {1:'PaisAccidente'})
l = l.drop([2], axis=1)
l['Fecha'] = pd.to_datetime(l['Fecha'])
l

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Newfoundland,Labradore Airways,1984-10-11,"Goose Bay, Newfoundland, Canada",de Havilland Canada DHC-6 Twin Otter 100
2,Colombia,LAC Colombia,1976-02-04,"Off Santa Maria, Colombia",Douglas DC-6A
3,Colombia,LAC Colombia,1978-04-29,"Bogota, Colombia",Douglas C-118A
4,Colombia,LAC Columbia,1982-02-19,"Cerro el Tablazo, Colombia",Douglas DC-6BF
5,Colombia,LACOL,1995-05-25,"Miraflores, Colombia",Douglas DC-3C
...,...,...,...,...,...
136,Bolivia,Lloyd Aéreo Boliviano,1966-08-03,"Benito, Bolivia",Curtiss C-46D
137,Bolivia,Lloyd Aéreo Boliviano,1976-10-13,"Santa Cruz, Bolivia",Boeing B-707-31
138,Colombia,Lloyd Aereo Colombiano,1960-04-19,"Bogota, Colombia",Curtiss C-46
139,Bolivia,Lloyd Aero Boliviano,1932-10-26,"Villa Monte, Bolivia",Ford Tri-motor 5


In [942]:
l2 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Lo-Lz.htm')
l2 = l2[0]
l2 = l2.drop([0],axis=0)
l2 = l2.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
l22 = l2['LugarAccidente'].str.split(',', expand=True)
l22 = l22.drop([0],axis=1)
l2 = pd.concat([l22, l2], axis=1)
l2 = l2.rename( columns= {1:'PaisAccidente'})
l2 = l2.drop([2], axis=1)
l2['Fecha'] = pd.to_datetime(l2['Fecha'])
l2

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Newfoundland,Labradore Airways,1984-10-11,"Goose Bay, Newfoundland, Canada",de Havilland Canada DHC-6 Twin Otter 100
2,Colombia,LAC Colombia,1976-02-04,"Off Santa Maria, Colombia",Douglas DC-6A
3,Colombia,LAC Colombia,1978-04-29,"Bogota, Colombia",Douglas C-118A
4,Colombia,LAC Columbia,1982-02-19,"Cerro el Tablazo, Colombia",Douglas DC-6BF
5,Colombia,LACOL,1995-05-25,"Miraflores, Colombia",Douglas DC-3C
...,...,...,...,...,...
136,Bolivia,Lloyd Aéreo Boliviano,1966-08-03,"Benito, Bolivia",Curtiss C-46D
137,Bolivia,Lloyd Aéreo Boliviano,1976-10-13,"Santa Cruz, Bolivia",Boeing B-707-31
138,Colombia,Lloyd Aereo Colombiano,1960-04-19,"Bogota, Colombia",Curtiss C-46
139,Bolivia,Lloyd Aero Boliviano,1932-10-26,"Villa Monte, Bolivia",Ford Tri-motor 5


In [943]:
m = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20M.htm')
m = m[0]
m = m.drop([0],axis=0)
m = m.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
m2 = m['LugarAccidente'].str.split(',', expand=True)
m2 = m2.drop([0],axis=1)
m = pd.concat([m2, m], axis=1)
m = m.rename( columns= {1:'PaisAccidente'})
m = m.drop([2,3], axis=1)
m['Fecha'] = pd.to_datetime(m['Fecha'])
m

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Philippines,Mabuhay Airways,1974-10-19,"Calapan, Philippines",Douglas DC-3
2,Australia,Mac Roberston-Miller Airlines,1949-07-02,"Near Guildford, Australia",Douglas C-47A-20-DL
3,Australia,Mac Robertson Miller Airlines,1945-06-24,"Gealdton, Australia",de Havilland DH-86A
4,Italy,Macchi,1938-08-18,"Montnago, Italy",Macchi C-94
5,WA,MacRobertson Miller Airlines,1968-12-31,"Port Hedland, WA, Australia",Vickers Viscount 720
...,...,...,...,...,...
1545,Angola,Zanex,1998-12-17,"Saurimo, Angola",Antonov 12
1546,Missouri,Zantop Air Transport,1954-01-20,"Kansas City, Missouri",Douglas DC-3A
1547,Colorado,Zantop Air Transport,1963-12-07,"Near Nederland, Colorado",Curtis C-46A-20-CU
1548,Michigan,Zantop Air Transport,1964-12-30,"Detroit, Michigan",Curtis C-46A


In [944]:
n = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20N-O.htm')
n = n[0]
n = n.drop([0],axis=0)
n = n.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
n2 = n['LugarAccidente'].str.split(',', expand=True)
n2 = n2.drop([0],axis=1)
n = pd.concat([n2, n], axis=1)
n = n.rename( columns= {1:'PaisAccidente'})
n = n.drop([2], axis=1)
n['Fecha'] = pd.to_datetime(n['Fecha'])
n

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Colombia,Nacional de Aviacion,2007-10-08,"Near Cubarral, Colombia",Let 410 UVP-E10A
2,Black Sea,Naftogaz,2008-04-28,"Off Crimea, Black Sea",Mil Mi-8T
3,Canada,Nahanni Air Services,1984-10-09,"Fort Franklin, Canada",de Havilland Canada DHC-6 Twin Otter 100
4,Russia,NARZ,2010-11-25,"Strezhevoy, Russia",Mil Mi-8T
5,Iceland,Naske Air,2001-08-05,"Reykjavík, Iceland",Dassault Falcon 20C
...,...,...,...,...,...
141,Philippines,Orient Air System and Integrated Services,1974-11-15,"Manila, Philippines",Douglas C-47A-30
142,India,Orient Airways,1953-03-14,"Kalasahar, India",Convair CV-240-7
143,United Arab Emirates,Orient Airways,1953-08-03,"Sharjah, United Arab Emirates",Douglas DC-3
144,Algeria,Oriental Airlines,1994-09-18,"Tamanrasset, Algeria",BAC One-Eleven 515FB


In [945]:
p = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Pa-Pd.htm')
p = p[0]
p = p.drop([0],axis=0)
p = p.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
p2 = p['LugarAccidente'].str.split(',', expand=True)
p2 = p2.drop([0],axis=1)
p = pd.concat([p2, p], axis=1)
p = p.rename( columns= {1:'PaisAccidente'})
p = p.drop([2], axis=1)
p['Fecha'] = pd.to_datetime(p['Fecha'])
p

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Philippines,Pacific Air,2004-10-16,"Coron Island, Philippines",Pilatus Britten-Norman BN-2A-21 Islander
2,California,Pacific Air Lines,1959-10-26,"Santa Maria, California",Douglas C-47A-DL
3,California,Pacific Air Lines,1964-05-07,"San Ramon, California",Fairchild F-27A
4,Oregon,Pacific Air Transport,1926-12-16,"Medford, Oregon",Ryan M-1
5,California,Pacific Air Transport,1930-11-18,"Techachapi Mountains, California",Boeing 40
...,...,...,...,...,...
135,Kuwait,PAS,1959-07-15,"Kuwait City, Kuwait",Avro 685 York
136,Vietnam,Pathet Lao Airlines,1974-03-08,"Hanoi, Vietnam",Antonov An-24
137,Morroco,Paukn Air,1998-09-25,"Near Nador, Morroco",British Aerospace BAe-146-100
138,California,Paul Kelly Flying Service Inc.,1965-11-14,"Near Palm Springs, California",Learjet 23


In [946]:
p2 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Pe-Pz.htm')
p2 = p2[0]
p2= p2.drop([0],axis=0)
p2 = p2.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
p22 = p2['LugarAccidente'].str.split(',', expand=True)
p22= p22.drop([0],axis=1)
p2 = pd.concat([p22, p2], axis=1)
p2 = p2.rename( columns= {1:'PaisAccidente'})
p2 = p2.drop([2], axis=1)
p2['Fecha'] = pd.to_datetime(p2['Fecha'])
p2

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Philippines,Pacific Air,2004-10-16,"Coron Island, Philippines",Pilatus Britten-Norman BN-2A-21 Islander
2,California,Pacific Air Lines,1959-10-26,"Santa Maria, California",Douglas C-47A-DL
3,California,Pacific Air Lines,1964-05-07,"San Ramon, California",Fairchild F-27A
4,Oregon,Pacific Air Transport,1926-12-16,"Medford, Oregon",Ryan M-1
5,California,Pacific Air Transport,1930-11-18,"Techachapi Mountains, California",Boeing 40
...,...,...,...,...,...
135,Kuwait,PAS,1959-07-15,"Kuwait City, Kuwait",Avro 685 York
136,Vietnam,Pathet Lao Airlines,1974-03-08,"Hanoi, Vietnam",Antonov An-24
137,Morroco,Paukn Air,1998-09-25,"Near Nador, Morroco",British Aerospace BAe-146-100
138,California,Paul Kelly Flying Service Inc.,1965-11-14,"Near Palm Springs, California",Learjet 23


In [947]:
r = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Q-R.htm')
r = r[0]
r= r.drop([0],axis=0)
r = r.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
r2 = r['LugarAccidente'].str.split(',', expand=True)
r2= r2.drop([0],axis=1)
r = pd.concat([r2, r], axis=1)
r = r.rename( columns= {1:'PaisAccidente'})
r = r.drop([2], axis=1)
r['Fecha'] = pd.to_datetime(r['Fecha'])
r

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Qld,Qantas,1927-03-24,"Tambo, Qld, Australia",de Havilland DH-9C
2,Australia,Qantas,1928-09-04,"Adelaide Hills, Australia",de Havilland DH.50J
3,Australia,Qantas,1934-10-03,"Near Winton, Australia",de Havilland DH-50A
4,Australia,Qantas,1934-11-15,"Near Longreach, Australia",de Havilland DH-86
5,Australia,Qantas,1938-12-12,"Darwin, Australia",Short S23 ‘C’ Class flying boat
...,...,...,...,...,...
86,Panama,Rutas Internacionales Peruanes,1965-08-04,"Panama City, Panama",Douglas C-54D
87,Rwanda,Rwanda Government,1994-04-06,"Kigali, Rwanda",Dassault Falcon 50
88,Rwanda,Rwandair,2009-11-12,"Kigali, Rwanda",Canadair CRJ-100ER
89,Alaska,Ryan Air Services,1987-11-23,"Homer, Alaska",Beechcraft 1900C


In [948]:
s = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Sa-Si.htm')
s = s[0]
s= s.drop([0],axis=0)
s = s.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
s2 = s['LugarAccidente'].str.split(',', expand=True)
s2= s2.drop([0],axis=1)
s = pd.concat([s2, s], axis=1)
s = s.rename( columns= {1:'PaisAccidente'})
s = s.drop([2], axis=1)
s['Fecha'] = pd.to_datetime(s['Fecha'])
s

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Malaysia,Sabah Air,1976-06-06,"Off Kota Kinabalu, Malaysia",GAF Nomad N-22B
2,Malaysia,Sabah Air,1995-11-29,"Off Labuan, Malaysia",Bell 412 (helicopter)
3,England,Sabena,1930-09-11,"Croydon, England",Sabca F-VII
4,Belgium,Sabena,1931-01-08,"Melle, Belgium",Fokker F-VII
5,England,Sabena,1935-12-10,"Croydon, England",Savoia-Marchetti SM73
...,...,...,...,...,...
154,Lancs,Silver City Airways,1958-02-27,"Horwich, Lancs, England",Bristol 170 Freighter 21
155,Guernsey,Silver City Airways,1961-11-01,"Les Prevosts, Guernsey",Bristol 170 Freighter 31M
156,Michigan,Simmons Airlines,1986-03-13,"Alpena, Michigan",Embraer 110P1 Bandeirante
157,Taiwan,Singapore Airlines,2000-10-31,"Taipei, Taiwan",Boeing B-747-412


In [949]:
s2 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Sj-Sz.htm')
s2 = s2[0]
s2= s2.drop([0],axis=0)
s2 = s2.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
s22 = s2['LugarAccidente'].str.split(',', expand=True)
s22= s22.drop([0],axis=1)
s2 = pd.concat([s22, s2], axis=1)
s2 = s2.rename( columns= {1:'PaisAccidente'})
s2 = s2.drop([2], axis=1)
s2['Fecha'] = pd.to_datetime(s2['Fecha'])
s2

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Sri Lanka,Sky Cabs,2000-03-24,"Kadirana, Sri Lanka",Antonov AN-12 (freighter)
2,None,Sky Dive New Zealand,2010-09-04,Fox Glacier Airstrip,Fletcher FU-24-954
3,Nigeria,Sky Executive Air Services,2002-05-21,"Near Calabar, Nigeria",LET 410UVP
4,Wyoming,Sky Train Air,1981-10-01,"Casper, Wyoming",Learjet 24
5,Missouri,Skycraft Air Transport,1984-01-09,"Saint Louis, Missouri",Douglas DC-3
...,...,...,...,...,...
149,Brazil,Syndicato Condor,1939-01-13,"Rio de Janeiro, Brazil",Junkers JU-52/3m
150,Syria,Syrian Air Force,2015-01-18,"Near Abu adh Dhuhur Air Base, Syria",Antonov An-26
151,Syria,Syrian Airways,1953-12-21,"Dah-el-Kadeeb, Syria",Douglas DC-3
152,Syria,Syrian Airways,1956-02-24,"Near Aleppo, Syria",Douglas DC-3


In [950]:
t = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Ta-Tq.htm')
t = t[0]
t= t.drop([0],axis=0)
t = t.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
t2 = t['LugarAccidente'].str.split(',', expand=True)
t2= t2.drop([0],axis=1)
t = pd.concat([t2, t], axis=1)
t = t.rename( columns= {1:'PaisAccidente'})
t = t.drop([2], axis=1)
t['Fecha'] = pd.to_datetime(t['Fecha'])
t

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Australia,TAA,1954-10-31,"Mangalore, Australia",Vickers 720 Viscount
2,Angola,TAAG Angola Airlines,1981-05-16,"Near Menongue, Angola",Lockheed L-100-20 Hercules
3,Angola,TAAG Angola Airlines,1982-11-29,"Lubango, Angola",Antonov AN-26
4,Huila,TAAG Angola Airlines,1983-11-08,"Lubango, Huila, Angola",Boeing B-737-200
5,Democratic Republic Congo,TAAG Angola Airlines,1997-04-14,"Brazzaville, Democratic Republic Congo",Fokker F-27 Friendship 600
...,...,...,...,...,...
119,England,Titan Airways,1993-01-13,"Sellafield, England",Embraer 110P1 Bandeirante
120,Zaire,TMK Air Commuter,1997-09-12,"Near Uvira, Zaire",de Havilland Canada DHC-6 Twin Otter 300
121,Japan,Toa Domestic Airline (Japan),1971-07-03,"Hokkaido, Japan",NAMC YS-11A-227
122,Sao Paulo,Total Air Lines,2002-09-14,"Paranapanema, Sao Paulo, Brazil",ATR 42-300


In [951]:
t2 = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20Tr-Tz.htm')
t2 = t2[0]
t2= t2.drop([0],axis=0)
t2 = t2.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
t22 = t2['LugarAccidente'].str.split(',', expand=True)
t22= t22.drop([0],axis=1)
t2 = pd.concat([t22, t2], axis=1)
t2 = t2.rename( columns= {1:'PaisAccidente'})
t2 = t2.drop([2,3], axis=1)
t2['Fecha'] = pd.to_datetime(t2['Fecha'])
t2

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Florida,Aaxico Airlines,1955-04-02,"Off Pensacola, Florida",Curtiss C-46A
2,Texas,Aaxico Airlines,1959-09-02,"Abilene, Texas",Curtiss C-46A
3,Washington,Aaxico Airlines,1965-04-23,"Near Seattle, Washington",Douglas DC-6A
4,None,AB Aerotransport,1943-08-27,North Sea,Douglas DC-3
5,Sweden,AB Aerotransport,1943-10-22,"Hallo, Sweden",Douglas DC-3
...,...,...,...,...,...
456,Nicaragua,Aerovias Guest,1956-11-15,"Puerto Somoza, Nicaragua",Douglas C-54A-10-DC
457,Argentina,Aerovias Halcon,1966-08-17,"Puertos Lobos, Argentina",Curtiss C-46F-1-CU
458,Mexico,Aerovias Rojas,1968-04-10,"Near Mexico City, Mexico",Douglas DC-3
459,Panama,Aerovias Urraca,1973-02-21,"Cerro Horqueta, Panama",Douglas DC-3


In [952]:
u = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20U.htm')
u = u[0]
u= u.drop([0],axis=0)
u = u.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
u2 = u['LugarAccidente'].str.split(',', expand=True)
u2= u2.drop([0],axis=1)
u = pd.concat([u2, u], axis=1)
u = u.rename( columns= {1:'PaisAccidente'})
u = u.drop([2], axis=1)
u['Fecha'] = pd.to_datetime(u['Fecha'])
u

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,None,U. S. Air Force,1944-09-18,Alaska,Douglas C-47A-90-DL
2,Newfoundland,u.s. Air Force,1953-03-18,"Stephenville, Newfoundland",Boeing B-29 Superfortress
3,Afghanistan,U.S. Government (leased),2006-04-24,"Lashkar Gah, Afghanistan",Antonov AN-32B
4,Greenland,U.S. Overseas Airways,1957-05-13,"Narsarressuak, Greenland",Douglas DC-4
5,Italy,Uganda Airlines,1988-10-17,"Rome, Italy",Boeing B-707-338C
...,...,...,...,...,...
315,Angola,Zanex,1998-12-17,"Saurimo, Angola",Antonov 12
316,Missouri,Zantop Air Transport,1954-01-20,"Kansas City, Missouri",Douglas DC-3A
317,Colorado,Zantop Air Transport,1963-12-07,"Near Nederland, Colorado",Curtis C-46A-20-CU
318,Michigan,Zantop Air Transport,1964-12-30,"Detroit, Michigan",Curtis C-46A


In [953]:
z = pd.read_html('http://www.planecrashinfo.com/Airline/AL%20V-Z.htm')
z = z[0]
z= z.drop([0],axis=0)
z = z.rename(columns={0:'OperadorAerolinea',1:'Fecha',2:'LugarAccidente',3:'TipoAeronave'})
z2 = z['LugarAccidente'].str.split(',', expand=True)
z2= z2.drop([0],axis=1)
z = pd.concat([z2, z], axis=1)
z = z.rename( columns= {1:'PaisAccidente'})
z = z.drop([2], axis=1)
z['Fecha'] = pd.to_datetime(z['Fecha'])
z

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Curacao,Valley Air Service,1977-12-24,"Willemstad, Curacao, Netherlands Antilles",Piper PA-31-350 Navajo
2,Miami,ValuJet,1996-05-11,"Everglades, Miami, Florida",McDonnell Douglas DC-9-32
3,Vanuatu,Vanair,1991-07-25,"Espiritu Santos, Vanuatu",Britten-Norman BN-2A-6 Islander
4,Vanuatu,Vanair,1999-05-08,"Off Port Vila, Vanuatu",de Havilland Canada DHC-6 Twin Otter 300
5,Brazil,Varig,1944-06-20,"Porto Alegre, Brazil",Lockheed 10C Electra
...,...,...,...,...,...
160,Angola,Zanex,1998-12-17,"Saurimo, Angola",Antonov 12
161,Missouri,Zantop Air Transport,1954-01-20,"Kansas City, Missouri",Douglas DC-3A
162,Colorado,Zantop Air Transport,1963-12-07,"Near Nederland, Colorado",Curtis C-46A-20-CU
163,Michigan,Zantop Air Transport,1964-12-30,"Detroit, Michigan",Curtis C-46A


In [954]:
df3 = pd.concat([ a, a1, a2, a3, a4, b, c, c2, d, f, h, j, l, l2, m, n, p, p2, r, s,s2, t,t2,u,z], axis=0)
df3

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
1,Russia,AKF Polet,1991-06-26,"Gulf of Finland, Russia",Antonov AN-24
2,Turkmenistan,Akhal Air,1996-03-12,"Karabogazgol Bay, Turkmenistan",Mil Mi-8T (helicopter)
3,Italy,Ala Littolia,1940-12-24,"Capri Island, Italy",Savoia-Marchetti SM.75
4,Egypt,Ala Littoria,1935-08-07,"Cairo, Egypt",Savoia-Marchetti S-66
5,Sudan,Ala Littoria,1937-08-02,"Wadi Halfa, Sudan",Savoia-Marchetti SM73
...,...,...,...,...,...
160,Angola,Zanex,1998-12-17,"Saurimo, Angola",Antonov 12
161,Missouri,Zantop Air Transport,1954-01-20,"Kansas City, Missouri",Douglas DC-3A
162,Colorado,Zantop Air Transport,1963-12-07,"Near Nederland, Colorado",Curtis C-46A-20-CU
163,Michigan,Zantop Air Transport,1964-12-30,"Detroit, Michigan",Curtis C-46A


Resumen hasta ahora:
EDA 3 databases:
- Accidentes aviones = df
- Causas Accidentes = df2
- Aerolineas Accidentes = df3

In [955]:
df = df.drop(['Id','summary'],axis=1)
df

,PaisAccidente,Fecha,LugarAccidente,OperadorAerolinea,TipoAeronave,Matricula,SerieConstruccion,TotalABordo,PasajerosABordo,TripulacionABordo,CantidadFallecidos,MuertesPasajeros,MuertesTripulacion,Tierra
31,New Jersey,1919-10-30,"Long Valley, New Jersey",US Aerial Mail Service,De Havilland DH.4,77,77,1,0,1,1,0,1,0
45,France,1920-10-02,"Off Port Vendres, France",Latecoere Airlines,Salmson 2-A-2,F-ALAI,31,2,1,1,2,1,1,0
46,Spain,1920-10-05,"Valencia, Spain",Latecoere Airlines,Breguet 14,F-ALTA,20,2,1,1,2,1,1,0
49,England,1920-12-14,"London, England",Handley Page Transport,Handley Page HP-16,G-EAMA,HP-25,8,6,2,4,2,2,0
50,Spain,1920-12-24,"Barcelona, Spain",Aeropostale,Breguet 14,F-ALBO,57,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5002,Sudan,2021-03-02,"Pieri, Sudan",South Sudan Supreme Airlines,Let L-410UVP-E,HK-4274,902525,10,8,2,10,8,2,0
5003,Alaska,2021-03-28,"Near Butte, Alaska",Soloy Helicopters,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0
5004,Nigeria,2021-05-21,"Near Kaduna, Nigeria",Military - Nigerian Air Force,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0
5005,Myanmar,2021-06-10,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0


In [956]:
df.reset_index()

,index,PaisAccidente,Fecha,LugarAccidente,OperadorAerolinea,TipoAeronave,Matricula,SerieConstruccion,TotalABordo,PasajerosABordo,TripulacionABordo,CantidadFallecidos,MuertesPasajeros,MuertesTripulacion,Tierra
0,31,New Jersey,1919-10-30,"Long Valley, New Jersey",US Aerial Mail Service,De Havilland DH.4,77,77,1,0,1,1,0,1,0
1,45,France,1920-10-02,"Off Port Vendres, France",Latecoere Airlines,Salmson 2-A-2,F-ALAI,31,2,1,1,2,1,1,0
2,46,Spain,1920-10-05,"Valencia, Spain",Latecoere Airlines,Breguet 14,F-ALTA,20,2,1,1,2,1,1,0
3,49,England,1920-12-14,"London, England",Handley Page Transport,Handley Page HP-16,G-EAMA,HP-25,8,6,2,4,2,2,0
4,50,Spain,1920-12-24,"Barcelona, Spain",Aeropostale,Breguet 14,F-ALBO,57,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4082,5002,Sudan,2021-03-02,"Pieri, Sudan",South Sudan Supreme Airlines,Let L-410UVP-E,HK-4274,902525,10,8,2,10,8,2,0
4083,5003,Alaska,2021-03-28,"Near Butte, Alaska",Soloy Helicopters,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0
4084,5004,Nigeria,2021-05-21,"Near Kaduna, Nigeria",Military - Nigerian Air Force,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0
4085,5005,Myanmar,2021-06-10,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0


In [957]:
df['PasajerosABordo'].iloc[145]

'4'

In [958]:
df[df['TotalABordo'] == 'NC12221']


,PaisAccidente,Fecha,LugarAccidente,OperadorAerolinea,TipoAeronave,Matricula,SerieConstruccion,TotalABordo,PasajerosABordo,TripulacionABordo,CantidadFallecidos,MuertesPasajeros,MuertesTripulacion,Tierra


In [959]:
df['LugarAccidente'] = df['LugarAccidente'].replace({',':'-'}, regex=True)
df.drop(columns=['TotalABordo'], axis=1, inplace=True)
df.drop(columns=['Matricula'], axis=1, inplace=True)
df.drop(columns=['SerieConstruccion'], axis=1, inplace=True)
df.drop(columns=['Tierra'], axis=1, inplace=True)
df.drop(columns=['LugarAccidente'], axis=1, inplace=True)
df2['LugarAccidente'] = df2['LugarAccidente'].replace({',':'-'}, regex=True)
df3['LugarAccidente'] = df3['LugarAccidente'].replace({',':'-'}, regex=True)

In [960]:
#quitamos caracteres especiales
df = df.replace({',':'.'}, regex=True)
df = df.replace({'.,':'.'}, regex=True)
df2 = df2.replace({',':'.'}, regex=True)
df2 = df2.replace({'.,':'.'}, regex=True)
df2 = df2.replace({'-':''}, regex=True)
df3 = df3.replace({',':''}, regex=True)
df2['Fecha'] = pd.to_datetime(df2['Fecha'])

In [961]:
df2

,PaisAccidente,Fecha,LugarAccidente,Operador/Aerolinea,CausaAccidente
1,Virginia,1947-06-13,Leesburg Virginia,Pennsylvania AL,Errores Control Trafico Aereo
2,Russia,1948-05-22,Khabarovsk Russia,Aeroflot,Errores Control Trafico Aereo
3,Virginia,1949-11-01,Arlington Virginia,Eastern Air Lines,Errores Control Trafico Aereo
4,Russia,1952-10-05,Skvoritsy Russia,Aeroflot,Errores Control Trafico Aereo
5,Spain,1958-04-14,Castel de Fels Spain,Aviaco,Errores Control Trafico Aereo
...,...,...,...,...,...
112,Russia,2004-08-24,RostovonDon Russia,Sibir Airlines,Sabotaje Secuestro
113,Ukraine,2014-07-17,Hrabove Ukraine,Malaysia Airlines,Sabotaje Secuestro
114,Fran,2015-03-24,DigneLesBains Fran,Germanwings,Sabotaje Secuestro
115,Egypt,2015-10-31,Hasna Egypt,Metrojet,Sabotaje Secuestro


In [962]:
df3.iloc[19]

PaisAccidente                             Domincan Republic
OperadorAerolinea    Alas Nacionales leased from Birgen Air
Fecha                                   1996-02-06 00:00:00
LugarAccidente          Off Puerto Plata- Domincan Republic
TipoAeronave                               Boeing B-757-225
Name: 20, dtype: object

In [963]:
#cambiamos orden de fecha para mysql
#tiempo = df2['Fecha'].str.split('/', expand=True)
#tiempo['Fecha1'] = tiempo[2].str.cat(tiempo[1], sep='-')
#tiempo
#tiempo['Fecha1'] = tiempo['Fecha1'].str.cat(tiempo[0], sep='-')
#tiempo.drop(columns=[0,1,2], axis=1, inplace=True)
#df2 = pd.concat([df2, tiempo], axis=1)
#df2.drop(columns=['Fecha'], axis=1, inplace=True)
#df2 = df2.rename(columns= {'Fecha1':'Fecha'})
#df2


In [964]:
#convertimos a csv para pasar a mysql
df.to_csv('DB_Accidentes_aviones.csv',sep=",",encoding='utf-8',index = None) 
df2.to_csv('DB_Causas_Accidentes.csv',sep=",",encoding='utf-8',index = None)
df3.to_csv('DB_Aerolineas.csv',sep=",",encoding='utf-8',index = None)

In [965]:
df6 = pd.read_csv(r'DB_Aerolineas.csv')
df6

,PaisAccidente,OperadorAerolinea,Fecha,LugarAccidente,TipoAeronave
0,Russia,AKF Polet,1991-06-26,Gulf of Finland- Russia,Antonov AN-24
1,Turkmenistan,Akhal Air,1996-03-12,Karabogazgol Bay- Turkmenistan,Mil Mi-8T (helicopter)
2,Italy,Ala Littolia,1940-12-24,Capri Island- Italy,Savoia-Marchetti SM.75
3,Egypt,Ala Littoria,1935-08-07,Cairo- Egypt,Savoia-Marchetti S-66
4,Sudan,Ala Littoria,1937-08-02,Wadi Halfa- Sudan,Savoia-Marchetti SM73
...,...,...,...,...,...
6059,Angola,Zanex,1998-12-17,Saurimo- Angola,Antonov 12
6060,Missouri,Zantop Air Transport,1954-01-20,Kansas City- Missouri,Douglas DC-3A
6061,Colorado,Zantop Air Transport,1963-12-07,Near Nederland- Colorado,Curtis C-46A-20-CU
6062,Michigan,Zantop Air Transport,1964-12-30,Detroit- Michigan,Curtis C-46A


In [966]:
df.iloc[[142]]

,PaisAccidente,Fecha,OperadorAerolinea,TipoAeronave,PasajerosABordo,TripulacionABordo,CantidadFallecidos,MuertesPasajeros,MuertesTripulacion
260,Illinois,1931-09-14,Chicago-Detroit Airways,Lockheed Vega,0,1,1,0,1
